# Check Goat coordinates
## Check goat coordinates with data provided by Goat genome project
Test coordinates received from goat genome consortium
## Working on CHI1
Setting up stuff

In [1]:
import csv
import json
import pprint
import random
import pandas
import functools

from collections import namedtuple, defaultdict
from ensemblrest import EnsemblRest
from Bio import Entrez
from Bio import SeqIO

from src.data.common import WORKING_ASSEMBLIES
from src.features.utils import get_project_dir, text_or_gzip_open, sanitize
from src.features.smarterdb import global_connection, VariantGoat

# Always tell NCBI who you are
Entrez.email = "paolo.cozzi@ibba.cnr.it"

# define ensemblrest helper
ensRest = EnsemblRest()

# define the source data file
data_dir = get_project_dir() / "data/external/GOA/CONSORTIUM"
capri_all_data = data_dir / "capri4dbsnp-base-CHI-ARS-OAR-UMD.csv.gz"

# make a connection to smarter database
global_connection()

# get the smarter assembly I want to test
CHI1 = WORKING_ASSEMBLIES['CHI1']

Defining some useful functions:

In [2]:
@functools.lru_cache(maxsize=1000)
def get_chr_from_ncbi(chrom_name: str):
    if chrom_name == 'null':
        return None
    
    if chrom_name.isdigit():
        return chrom_name
    
    # get data from NCBI
    with Entrez.efetch(db="nucleotide", id=chrom_name, rettype="gb", retmode="text") as handle:
        record = SeqIO.read(handle, "genbank")
        
    return record.features[0].qualifiers["chromosome"][0]


def check_coordinates_chi1(record, variant):
    check = True
    location = variant.locations[0]
    
    # ['chi_1_0_chr', 'chi_1_0_pos', 'chi_1_0_strand']
    if record.chi_1_0_chr != location.chrom:
        check = False
        
    if record.chi_1_0_pos != location.position:
        check = False
        
    return check


def search_mismatches(datafile, smarter_assembly, check_coordinates):    
    # track mismatches in positions
    mismatches = []

    # track missing sps
    missing = []

    with text_or_gzip_open(datafile) as handle:
        reader = csv.reader(handle, delimiter="\t")
        header = next(reader)
        header = [sanitize(col) for col in header]
        Record = namedtuple("Record", header)
        for line in reader:
            # fix position columns
            for field in filter(lambda field: 'pos' in field, header):
                try:
                    idx = header.index(field)
                    line[idx] = int(line[idx])

                except ValueError as exc:
                    continue

            # fix chi1 chrom names
            idx = header.index('chi_1_0_chr')
            line[idx] = get_chr_from_ncbi(line[idx])

            # fix ars1 chrom_name
            idx = header.index('ars1_chr')
            line[idx] = get_chr_from_ncbi(line[idx])
            
            # make a record from csv line
            record = Record._make(line)

            # get variant and chosen location from database
            try:
                variant = VariantGoat.objects(name=record.locus_name).fields(
                    pk=0, rs_id=1, name=1, elemMatch__locations=smarter_assembly._asdict()).get()
            except VariantGoat.DoesNotExist:
                # print(f"Couldn't find variant {record.locus_name}")
                missing.append(record)
                continue

            # check data
            if not check_coordinates(record, variant):
                # print(f"Variant is different: {record} <> {variant}, {variant.locations[0]}")
                mismatches.append([record, variant])

    # print(record)
    # pprint.pprint(json.loads(variant.to_json()))
    print(f"There are {len(mismatches)} mismatches")
    print(f"{len(missing)} SNPs are not in database")
    
    return mismatches, missing

Now check *CHI1* coordinates:

In [3]:
mismatches_chi1, missing_chi1 = search_mismatches(capri_all_data, CHI1, check_coordinates_chi1)

There are 1420 mismatches
6653 SNPs are not in database


In [4]:
random.seed(a=42, version=2)
selected_chi1 = random.sample(mismatches_chi1, 20)

In [5]:
data = defaultdict(list)

for record, variant in selected_chi1:
    data["locus_name"].append(record.locus_name)
    data["rs_"].append(record.rs_)
    data["chi_1_0_chr"].append(record.chi_1_0_chr)
    data["chi_1_0_pos"].append(record.chi_1_0_pos)
    data["rs_id"].append(variant.rs_id)
    data["smarter_chrom"].append(variant.locations[0].chrom)
    data["smarter_pos"].append(variant.locations[0].position)

pandas.DataFrame.from_dict(data)

,locus_name,rs_,chi_1_0_chr,chi_1_0_pos,rs_id,smarter_chrom,smarter_pos
0,snp53417-scaffold807-74773,rs268285051,Unknown,74773,rs268285051,0,0
1,snp10135-scaffold1361-44576,rs268243008,Unknown,44576,rs268243008,0,0
2,snp2042-scaffold1061-20259,rs268235116,Unknown,20259,rs268235116,0,0
3,snp21860-scaffold2144-53078,rs268254327,Unknown,53078,rs268254327,0,0
4,snp19559-scaffold1971-21103,rs268252089,Unknown,21103,rs268252089,0,0
5,snp18430-scaffold1867-78532,rs268251028,Unknown,78532,rs268251028,0,0
6,snp13189-scaffold1505-57602,rs268245970,Unknown,57602,rs268245970,0,0
7,snp6785-scaffold1249-70584,rs268239737,Unknown,70584,rs268239737,0,0
8,snp59632-scaffold985-45095,rs268291047,Unknown,45095,rs268291047,0,0
9,snp37483-scaffold4582-6164,rs268269564,Unknown,6164,rs268269564,0,0


Coordinates seems to match with database for *CHI1* assembly. Differences are in contigs (unknown chromosomes)

## Working with ARS1 coordinates
Checking latest coordinates version

In [6]:
# get the smarter assembly I want to test
ARS1 = WORKING_ASSEMBLIES['ARS1']

# defining a new function to test coordinates
def check_coordinates_ars1(record, variant):
    check = True
    location = variant.locations[0]
    
    # ['ars1_chr', 'ars1_pos', 'ars1_strand']
    if record.ars1_chr != location.chrom:
        check = False
        
    if record.ars1_pos != location.position:
        check = False
        
    return check

Search mismatches in *ARS1* assembly:

In [7]:
mismatches_ars1, missing_ars1 = search_mismatches(capri_all_data, ARS1, check_coordinates_ars1)

There are 1184 mismatches
6653 SNPs are not in database


In [8]:
random.seed(a=42, version=2)
selected_ars1 = random.sample(mismatches_ars1, 20)

# get rs_id from selected items
selected_rsID = [variant.rs_id for (record, variant) in selected_ars1 if variant.rs_id]

# get informations from ensembl
result_ars1 = ensRest.getVariationByMultipleIds(ids=selected_rsID, species="capra_hircus")

In [11]:
def to_pd_table(selected, result):
    data = defaultdict(list)

    for record, variant in selected:
        data["entry"].append(record.locus_name)
        data["rs_"].append(record.rs_)
        data["chrom"].append(record.ars1_chr)
        data["pos"].append(record.ars1_pos)
        data["rs_id"].append(variant.rs_id)
        data["smarter_chrom"].append(variant.locations[0].chrom)
        data["smarter_pos"].append(variant.locations[0].position)

        if variant.rs_id and not 'failed' in result[variant.rs_id]:
            data["ensembl_chrom"].append(result[variant.rs_id]['mappings'][0]['seq_region_name'])
            data["ensembl_pos"].append(result[variant.rs_id]['mappings'][0]['start'])
        
        else:
            data["ensembl_chrom"].append(None)
            data["ensembl_pos"].append(None)
        
        if 'failed' in result[variant.rs_id]:
            data["ensembl_failed"].append(result[variant.rs_id]['failed'])
        
        else:
            data["ensembl_failed"].append(None)

    return pandas.DataFrame.from_dict(data)

to_pd_table(selected_ars1, result_ars1)

,entry,rs_,chrom,pos,rs_id,smarter_chrom,smarter_pos,ensembl_chrom,ensembl_pos,ensembl_failed
0,snp10978-scaffold13932-19256,rs268243822,18,64350718,rs268243822,18,64495025,None,NaN,Variant does not map to the genome
1,snp2720-scaffold1079-655080,rs268235776,Unknown,20183,rs268235776,0.NW_scaffold,20183,7,92001864.0,None
2,snp24769-scaffold2530-14952,rs268257162,Unknown,36932,rs268257162,0.NW_scaffold,9829,None,NaN,Variant does not map to the genome
3,snp22886-scaffold228-4649518,rs268255332,Unknown,372155,rs268255332,0.NW_scaffold,372155,None,NaN,Variant does not map to the genome
4,snp21494-scaffold21-244274,rs268253976,14,-1,rs268253976,null,0,None,NaN,Variant does not map to the genome
5,snp13706-scaffold153-960292,rs268246476,Unknown,858874,rs268246476,0.NW_scaffold,858874,None,NaN,Variant does not map to the genome
6,snp8812-scaffold1316-38259,rs268241723,Unknown,160485,rs268241723,0.NW_scaffold,160485,None,NaN,Variant does not map to the genome
7,snp58664-scaffold956-522857,rs268290160,13,71181569,rs268290160,13,71187835,None,NaN,Variant maps to more than one genomic location
8,snp7608-scaffold1271-347413,rs268240542,Unknown,146208,rs268240542,0.NW_scaffold,146208,None,NaN,Variant does not map to the genome
9,snp37951-scaffold47-1337018,rs268270029,Unknown,57698,rs268270029,0.NW_scaffold,57698,None,NaN,Variant does not map to the genome


There are few mismatches from received coordinates and the data I have in the database. However, the smarter data comes from the latest chip so I belive I can trust it. There are some considerations to made about SNP placement: I thick I need to cross-check with ensembl and try to align the probe myself to understand if there are issues on chip placement